Imports

In [1]:
import pandas as pd
import numpy as np
import sys
import os
import re

from src.image_tools import ImageTools
from src.load_image import load_image_to_numpy_array
pd.options.mode.chained_assignment = None  # default='warn'

Paths and constants

In [2]:

WORKING_DIR = "C:/Users/Sigurd/OneDriveMS/FYS-3741-MASTER/"
GT_PATH = WORKING_DIR+"data/data_yoloformat/test/annotations_1class/"

IMAGE_PATH = WORKING_DIR + "data/data_yoloformat/test/images/"
#yolo
PRED_PATH = WORKING_DIR+"results/yolov4_testresults/1class/yolo_format/"
#PRED_PATH = WORKING_DIR+"results/yolov4_testresults/annotations/absolute_format/"
#faster-rcnn
#PRED_PATH = WORKING_DIR+"results/fasterrcnn_testresults/yolo_format/"
#PRED_PATH = WORKING_DIR+"results/efficientDet_testresults_0.1thresh/abs_format/"
#PRED_PATH = WORKING_DIR + 'results/fasterrcnn_testresults/yolo_format/'


#WORKING_DIR = os.getcwd()


#GT_PATH = "examples/test_labels/"
GT = os.listdir(GT_PATH)
#IMAGE_PATH = "examples/test_im/"
IM = os.listdir(IMAGE_PATH)
#PRED_PATH = "examples/test_pred/"
PRED = os.listdir(PRED_PATH)

In [5]:



class EvaluationTools:
    def __init__(self, gt_path, im_path, pred_path, iou_threshold):
        self.im_path = im_path
        self.gt_path = gt_path
        self.pred_path = pred_path
        self.iou_threshold = iou_threshold
        self.names = ['classification', 'x', 'y', 'w', 'h']
        self.names_pred = ['classification', 'confidence', 'x', 'y', 'w', 'h']

        self.im_w, self.im_h = self.get_image_info()


    def get_bbox(self):
        """
        Convert and return bonding box from path to pandas df (in x, y, h, w)
        """
        gt = pd.read_csv(self.gt_path, sep = ' ', names = self.names, index_col = False)
        
        pred = pd.read_csv(self.pred_path, sep = ' ', header=None).drop(6, axis=1)
        pred.columns = self.names_pred
        #print('get bbox gt: ', gt)
        return pred, self.yolo_to_absolute(gt)


    def absolute_to_yolo(self, bbox):
        """
        NOT USED
        Convert bbox format from (xmin, ymin, w, h) to yolo
        """
        n = (bbox.shape[0])
        converted_bbox = bbox.copy()
        for i in range(n):
            converted_bbox['x'][i] = bbox['x'][i] / self.im_w
            converted_bbox['y'][i] = bbox['y'][i] / self.im_h
            converted_bbox['w'][i] = bbox['w'][i] / self.im_w
            converted_bbox['h'][i] = bbox['h'][i] / self.im_h

        return converted_bbox

    def absolute_to_coordinates(self, bbox):
        """
        convert bbox format from (xmin, ymin, w, h) to (xmin, ymin, xmax, ymax).
        Param: bbox:
                numpy array shape [n,4]
        returns:
                numpy array shape [n,4]
        """
        converted_bbox = bbox.copy()
        converted_bbox[:, 2] = bbox[:, 0] + bbox[:, 2]
        converted_bbox[:, 3] = bbox[:, 1] + bbox[:, 3]

        return converted_bbox

    def yolo_to_absolute(self, bbox):
        """
        Convert bbox format from yolo to (xmin, ymin, w, h)
        """
        n = (bbox.shape[0])
        converted_bbox = bbox.copy()
        #print('yolo_to_absolute bbox: ', bbox)
        for i in range(n):
            converted_bbox['x'][i] = (bbox['x'][i] - 0.5*bbox['w'][i]) * self.im_w
            converted_bbox['y'][i] = (bbox['y'][i] - 0.5*bbox['h'][i]) * self.im_h
            converted_bbox['w'][i] = bbox['w'][i] * self.im_w
            converted_bbox['h'][i] = bbox['h'][i] * self.im_h

        return converted_bbox

    def get_image_info(self):
        """
        Fetch image height and width
        """
        im = load_image_to_numpy_array(self.im_path)
        im_w = im.shape[1]
        im_h = im.shape[0]
        return im_w, im_h
    
    def get_iou(self, pred, gt):
        """
        Calculate intersection over union (IoU) between all bounding boxes in image.
        Also calculates total pixel area of each bounding box

        param:
            
        returns:
        iou: float(0,1)
        area_bb1: float
        area_bb2: float
        """
        #print('pred:', pred)
        #print('gt: ',gt)
        #remove values class/conf values and converte to coordinate form (pascalVOC)
        bb1 = self.absolute_to_coordinates(np.array(pred)[:, 2:])
        #print(bb1)
        #add extra axis for calculation
        bb1 = bb1[:, None]
        #remove class value and convert to coordinate form (pascalVOC)
        bb2 = self.absolute_to_coordinates(np.round(np.array(gt)[:, 1:], 0))
        #print(bb2)
        #print(bb1)
        #print(bb2)
        #calculation...
        low = np.s_[...,:2]
        high = np.s_[..., 2:]

        bb1[high] += 1; bb2[high] += 1
        
        #intersect
        intrs = (np.maximum(0,np.minimum(bb1[high],bb2[high])
                            -np.maximum(bb1[low],bb2[low]))).prod(-1)
        #iou
        area_pred = (bb1[high]-bb1[low]).prod(-1)
        area_gt = (bb2[high]-bb2[low]).prod(-1)
        iou = intrs / (area_pred + area_gt -intrs + 1e-16)

        return iou, area_gt
    
    def find_valid_detections(self, iou, iou_threshold, index_list):
        """
        find number TP, FP, FN 
        """
        #print(iou)
        #iou = np.zeros(iou_0.shape)
        #print(iou.shape)
        closest_box = np.max(iou, axis=1)
        closest_gt = np.max(iou, axis=0)
        valid_boxes = closest_box.copy()
        valid_boxes[np.where(closest_box <= iou_threshold)] = 0
        valid_gt = closest_gt.copy()
        valid_gt[np.where(closest_gt <= iou_threshold)] = 0


        tp = np.zeros(len(index_list)); fp = np.zeros(len(index_list)); fn = np.zeros(len(index_list))

        for i in range(len(index_list)):
            try:
                tp[i] += np.count_nonzero(valid_boxes[index_list[i]])
                fp[i] += closest_box[index_list[i]].shape[0] - np.count_nonzero(valid_boxes[index_list[i]])
            except:
                tp[i] += np.count_nonzero(valid_gt[index_list[i]])
                fn[i] = closest_gt[index_list[i]].shape[0] - np.count_nonzero(valid_gt[index_list[i]])
                 
        return tp, fp, fn
        
        
    
    
    def number_of_detections(self, area_gt):

        area = np.ravel(area_gt)
        #print(area)
        small_thresh = 32**2
        med_thresh = 96**2
        
        index_full = np.full(area.shape, True)
        index_small = np.full(area.shape, False)
        index_med = np.full(area.shape, False)
        index_large = np.full(area.shape, False)
        
        #print('index_small:', index_small)
        
        for i in range(area.shape[0]):
           
            if area[i] < small_thresh:
                index_small[i] = True
                #pred_small += 1
            elif area[i] >= small_thresh and area[i] < med_thresh:
                index_med[i] = True
                #pred_med += 1
            elif area[i] >= med_thresh:
                index_large[i] = True
                #pred_large += 1
            else:
                raise Exception('Value for bbox area is invalid')

        #print('index_small post loop:', index_small)
        #print([index_full, index_small, index_med, index_large])
        return [index_full, index_small, index_med, index_large]

    def no_predictions(self):
        """
        Handles situations where there are no valid predictions for a ground truth. Every gt object will be a false negative.
        """
        gt = pd.read_csv(self.gt_path, sep = ' ', names = self.names, index_col = False)
        gt = self.yolo_to_absolute(gt)
        low = np.s_[...,:2]
        high = np.s_[..., 2:]
        bb2 = self.absolute_to_coordinates(np.round(np.array(gt)[:, 1:], 0))
        area_gt = (bb2[high]-bb2[low]).prod(-1)
        indexes = self.number_of_detections(area_gt)

        fn = np.sum(np.array([element*1 for element in indexes]), axis=1)
        
        return fn

        
#main()


In [6]:
def recall(tp, fn):
    return tp / (tp + fn + 1e-10)


def precision(tp, fp):
    return tp / (tp + fp + 1e-10)
    

#def main():
TP = np.zeros(4)
FP = np.zeros(4)
FN = np.zeros(4)
PRED_LARGE = 0
PRED_MEDIUM = 0
PRED_SMALL = 0
PRED_TOTAL = 0
THRESH = 0.1

for test_index in range(len(PRED)):
#for test_index in range(1115, 1120):
    sys.stdout.write('\r Image: {}, completion: {}%'.format(test_index, np.round((test_index + 1) / len(IM) * 100, 2)))
    #for test_index in range(1):
    #test_index = 4
    #print(IMAGE_PATH+IM[test_index])
    #print(GT_PATH+GT[test_index])
    #print(PRED_PATH+PRED[test_index])
    txt = PRED[test_index]

    file_number = re.findall(r'\d+', txt)[0]

    test = EvaluationTools(gt_path = GT_PATH+'Label_images'+file_number+'.txt', im_path = IMAGE_PATH+'Label_images'+file_number+'.jpg',
                           pred_path=PRED_PATH+PRED[test_index], iou_threshold=THRESH)

    try:
        pred, gt = test.get_bbox()
        try:
            iou, area_pred = test.get_iou(pred, gt)
            area_indexes = test.number_of_detections(area_pred)
            #n_tot = len(area_indexes)
            #n_small = sum(area_indexes[1])

            tp, fp, fn = test.find_valid_detections(iou, THRESH, area_indexes)
        except:
            print('Something is wrong with the TP/FP/FN calculation at index {}'.format(IM[test_index]))
    except:
        tp = np.zeros(4)
        fp = np.zeros(4)
        fn = test.no_predictions()
        #print(fn)



    TP += tp
    FP += fp
    FN += fn
    sys.stdout.flush()


 Image: 0, completion: 0.05%[[754. 397. 857. 531.]]
[[761. 407. 851. 510.]]
(1, 1)
 Image: 1, completion: 0.09%[[761. 386. 856. 519.]]
[[762. 393. 847. 513.]]
(1, 1)
 Image: 2, completion: 0.14%[[772. 390. 876. 516.]]
[[783. 412. 866. 510.]]
(1, 1)
 Image: 3, completion: 0.18%[[785. 383. 894. 523.]]
[[788. 406. 881. 515.]]
(1, 1)
 Image: 4, completion: 0.23%[[799. 386. 903. 546.]]
[[802. 415. 895. 519.]]
(1, 1)
 Image: 5, completion: 0.28%[[809. 399. 905. 535.]]
[[813. 398. 899. 549.]]
(1, 1)
 Image: 6, completion: 0.32%

KeyboardInterrupt: 

In [ ]:
names = ['Total:', 'small:', 'medium:', 'large:']
for i in range(TP.shape[0]):
    print(names[i])
    print('precision: ',precision(TP[i], FP[i]))
    print('recall: ',recall(TP[i], FN[i]))

print('False Negatives: [total, small, medium, large]')
print(FN)
print('False Positives: [total, small, medium, large]')
print(FP)
print('True Positives: [total, small, medium, large]')
print(TP)

Total:
precision:  0.9987760097918972
recall:  0.9772455089820125
small:
precision:  0.99999999995
recall:  0.4999999999875
medium:
precision:  0.9959903769044911
recall:  0.9394856278365401
large:
precision:  0.9999999999999647
recall:  0.9954369954369605
False Negatives: [total, small, medium, large]
[95.  2. 80. 13.]
False Positives: [total, small, medium, large]
[5. 0. 5. 0.]
True Positives: [total, small, medium, large]
[4.080e+03 2.000e+00 1.242e+03 2.836e+03]
